In [0]:
import sys
!git clone https://github.com/keras-team/keras-contrib.git
sys.path.append('/content/keras-contrib')

Cloning into 'keras-contrib'...
remote: Enumerating objects: 3622, done.
remote: Total 3622 (delta 0), reused 0 (delta 0), pack-reused 3622
Receiving objects: 100% (3622/3622), 858.42 KiB | 8.17 MiB/s, done.
Resolving deltas: 100% (2322/2322), done.


In [0]:
!git clone https://github.com/MrudulaSatya/night2day.git

Cloning into 'night2day'...
remote: Enumerating objects: 749, done.
remote: Total 749 (delta 0), reused 0 (delta 0), pack-reused 749
Receiving objects: 100% (749/749), 69.61 MiB | 44.41 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [0]:
!pip install h5py

In [0]:
import tensorflow as tf
import csv
import os.path
'''
class Logger(object):
    """Tensorboard logger."""

    def __init__(self, log_dir):
        """Initialize summary writer."""
        self.writer = tf.summary.FileWriter(log_dir)

    def scalar_summary(self, tag, value, step):
        """Add scalar summary."""
        summary = tf.Summary(value=[tf.Summary.Value(tag=tag, simple_value=value)])
        self.writer.add_summary(summary, step)
'''      
class CsvLogger:
    def __init__(self, filepath, filename="logn2d.csv", data=None):
        self.log_path = filepath
        self.log_name = filename
        self.csv_path = os.path.join(self.log_path, self.log_name)
        self.fieldsnames = ['D/loss', 'D/acc', 'G/loss']
        with open(self.csv_path, 'w') as f:
          writer = csv.DictWriter(f, fieldnames=self.fieldsnames)
          writer.writeheader()
    
    def scalar_summary(self, loss):
        data = {}
        for tag, value in loss.items():
          data[tag] = value
        if data is not None:
          self.write(data)

    def write(self, data):
        with open(self.csv_path, 'a') as f:
            writer = csv.DictWriter(f, fieldnames=self.fieldsnames)
            writer.writerow(data)
           


In [0]:
import scipy
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

class DataLoader():
    def __init__(self, dataset_name, img_res=(128, 128)):
        self.dataset_name = dataset_name
        self.img_res = img_res

    def load_data(self, batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "test"
        path = glob('./%s/%s/*' % (self.dataset_name, data_type))

        batch_images = np.random.choice(path, size=batch_size)

        imgs_A = []
        imgs_B = []
        for img_path in batch_images:
            img = self.imread(img_path)

            h, w, _ = img.shape
            _w = int(w/2)
            img_A, img_B = img[:, :_w, :], img[:, _w:, :]

            img_A = scipy.misc.imresize(img_A, self.img_res)
            img_B = scipy.misc.imresize(img_B, self.img_res)

            # If training => do random flip
            if not is_testing and np.random.random() < 0.5:
                img_A = np.fliplr(img_A)
                img_B = np.fliplr(img_B)

            imgs_A.append(img_A)
            imgs_B.append(img_B)

        imgs_A = np.array(imgs_A)/127.5 - 1.
        imgs_B = np.array(imgs_B)/127.5 - 1.

        return imgs_A, imgs_B

    def load_batch(self, batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "test"
        path = glob('./%s/%s/*' % (self.dataset_name, data_type))

        self.n_batches = int(len(path) / batch_size)

        for i in range(self.n_batches-1):
            batch = path[i*batch_size:(i+1)*batch_size]
            imgs_A, imgs_B = [], []
            for img in batch:
                img = self.imread(img)
                h, w, _ = img.shape
                half_w = int(w/2)
                img_A = img[:, :half_w, :]
                img_B = img[:, half_w:, :]

                img_A = scipy.misc.imresize(img_A, self.img_res)
                img_B = scipy.misc.imresize(img_B, self.img_res)

                if not is_testing and np.random.random() > 0.5:
                        img_A = np.fliplr(img_A)
                        img_B = np.fliplr(img_B)

                imgs_A.append(img_A)
                imgs_B.append(img_B)

            imgs_A = np.array(imgs_A)/127.5 - 1.
            imgs_B = np.array(imgs_B)/127.5 - 1.

            yield imgs_A, imgs_B


    def imread(self, path):
        return scipy.misc.imread(path, mode='RGB').astype(np.float)


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from __future__ import print_function, division
##os.environ['THEANO_FLAGS'] = "device=gpu1"    
#import theano
import scipy

from keras.datasets import mnist
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import sys

import numpy as np
import os


class Pix2Pix():
    def __init__(self):
        # Input shape
        self.img_rows = 256
        self.img_cols = 256
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        # Configure data loader
        self.dataset_name = 'night2day'
        self.data_loader = DataLoader(dataset_name=self.dataset_name,
                                      img_res=(self.img_rows, self.img_cols))

        print("dataloader done")
        print(self.data_loader)
        # Calculate output shape of D (PatchGAN)
        patch = int(self.img_rows / 2**4)
        self.disc_patch = (patch, patch, 1)

        #logging
        self.log_dir = "/content/gdrive/My Drive/"
        self.logger = CsvLogger(self.log_dir)
        
        # Number of filters in the first layer of G and D
        self.gf = 64
        self.df = 64

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

        #-------------------------
        # Construct Computational
        #   Graph of Generator
        #-------------------------

        # Build the generator
        self.generator = self.build_generator()

        # Input images and their conditioning images
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)

        # By conditioning on B generate a fake version of A
        fake_A = self.generator(img_B)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # Discriminators determines validity of translated images / condition pairs
        valid = self.discriminator([fake_A, img_B])

        self.combined = Model(inputs=[img_A, img_B], outputs=[valid, fake_A])
        self.combined.compile(loss=['mse', 'mae'],
                              loss_weights=[1, 100],
                              optimizer=optimizer)

    def build_generator(self):
        """U-Net Generator"""

        def conv2d(layer_input, filters, f_size=4, bn=True):
            """Layers used during downsampling"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if bn:
                d = BatchNormalization(momentum=0.8)(d)
            return d

        def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
            """Layers used during upsampling"""
            u = UpSampling2D(size=2)(layer_input)
            u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
            if dropout_rate:
                u = Dropout(dropout_rate)(u)
            u = BatchNormalization(momentum=0.8)(u)
            u = Concatenate()([u, skip_input])
            return u

        # Image input
        d0 = Input(shape=self.img_shape)

        # Downsampling
        d1 = conv2d(d0, self.gf, bn=False)
        d2 = conv2d(d1, self.gf*2)
        d3 = conv2d(d2, self.gf*4)
        d4 = conv2d(d3, self.gf*8)
        d5 = conv2d(d4, self.gf*8)
        d6 = conv2d(d5, self.gf*8)
        d7 = conv2d(d6, self.gf*8)

        # Upsampling
        u1 = deconv2d(d7, d6, self.gf*8)
        u2 = deconv2d(u1, d5, self.gf*8)
        u3 = deconv2d(u2, d4, self.gf*8)
        u4 = deconv2d(u3, d3, self.gf*4)
        u5 = deconv2d(u4, d2, self.gf*2)
        u6 = deconv2d(u5, d1, self.gf)

        u7 = UpSampling2D(size=2)(u6)
        output_img = Conv2D(self.channels, kernel_size=4, strides=1, padding='same', activation='tanh')(u7)

        return Model(d0, output_img)

    def build_discriminator(self):

        def d_layer(layer_input, filters, f_size=4, bn=True):
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if bn:
                d = BatchNormalization(momentum=0.8)(d)
            return d

        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)

        # Concatenate image and conditioning image by channels to produce input
        combined_imgs = Concatenate(axis=-1)([img_A, img_B])

        d1 = d_layer(combined_imgs, self.df, bn=False)
        d2 = d_layer(d1, self.df*2)
        d3 = d_layer(d2, self.df*4)
        d4 = d_layer(d3, self.df*8)

        validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

        return Model([img_A, img_B], validity)

    def train(self, epochs, batch_size=1, sample_interval=50):

        #print("inside traib function")
        start_time = datetime.datetime.now()

        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)

        for epoch in range(epochs):
          for batch_i, (imgs_A, imgs_B) in enumerate(self.data_loader.load_batch(batch_size)):
              #print("inside traib function")

                # ---------------------
                #  Train Discriminator
                # ---------------------

                # Condition on B and generate a translated version
              fake_A = self.generator.predict(imgs_B)

                # Train the discriminators (original images = real / generated = Fake)
              d_loss_real = self.discriminator.train_on_batch([imgs_A, imgs_B], valid)
              d_loss_fake = self.discriminator.train_on_batch([fake_A, imgs_B], fake)
              d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
              
              # Logging.
              loss = {}
              loss['D/loss'] = d_loss[0]
              loss['D/acc'] = d_loss[1]

                # -----------------
                #  Train Generator
                # -----------------

                # Train the generators
              g_loss = self.combined.train_on_batch([imgs_A, imgs_B], [valid, imgs_A])
              
              loss['G/loss'] = g_loss[0]

              elapsed_time = datetime.datetime.now() - start_time
                # Plot the progress
              #print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %f] time: %s" % (epoch, epochs,
                                                                        #batch_i, self.data_loader.n_batches,
                                                                        #d_loss[0], 100*d_loss[1],
                                                                        #g_loss[0],
                                                                        #elapsed_time))
              self.logger.scalar_summary(loss)

                # If at save interval => save generated image samples
              if batch_i % sample_interval == 0:
                  self.sample_images(epoch, batch_i)

    def test(self):
        self.testLogger = CsvLogger(self.log_dir, filename="logtestn2d.csv")
        os.makedirs('imagesTestRecn2d/%s' % self.dataset_name, exist_ok=True)
        r, c = 3, 3
        valid = np.ones((1,) + self.disc_patch)
        epoch = 0
        for batch_i, (imgs_A, imgs_B) in enumerate(self.data_loader.load_batch(batch_size=1,is_testing=True)):
          test_loss = self.combined.evaluate([imgs_A, imgs_B], [valid, imgs_A])
          loss_test = {}
          loss_test['G/loss'] = test_loss[0]
          self.testLogger.scalar_summary(loss_test)
          fake_A = self.generator.predict(imgs_B)
          gen_imgs = np.concatenate([fake_A, fake_A, fake_A])

          # Rescale images 0 - 
          gen_imgs = 0.5 * gen_imgs + 0.5

          titles = ['Generated', 'Generated', 'Original']
          fig, axs = plt.subplots(r, c)
          cnt = 0
          for i in range(r):
            for j in range(c):
              axs[i,j].imshow(gen_imgs[cnt])
              axs[i, j].set_title(titles[i])
              axs[i,j].axis('off')
              cnt += 1
              break
            break
          fig.savefig("imagesTestRecn2d/%d_%d_rec.png" % (epoch, batch_i))
          plt.close()
          self.sample_images1(epoch,batch_i)
          epoch += 1
    
    def sample_images1(self, epoch, batch_i):
        os.makedirs('imagesTRecAlln2d/%s' % self.dataset_name, exist_ok=True)
        r, c = 3, 3

        imgs_A, imgs_B = self.data_loader.load_data(batch_size=3, is_testing=True)
        fake_A = self.generator.predict(imgs_B)

        gen_imgs = np.concatenate([imgs_B, fake_A, imgs_A])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Condition', 'Generated', 'Original']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[i, j].set_title(titles[i])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("imagesTRecAlln2d/%d_%d_rec.png" % (epoch, batch_i))
        plt.close()


    def sample_images(self, epoch, batch_i):
        os.makedirs('imagesn2d/%s' % self.dataset_name, exist_ok=True)
        r, c = 3, 3

        imgs_A, imgs_B = self.data_loader.load_data(batch_size=3, is_testing=True)
        fake_A = self.generator.predict(imgs_B)

        gen_imgs = np.concatenate([imgs_B, fake_A, imgs_A])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Generated', 'Generated', 'Original']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[i, j].set_title(titles[i])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("imagesn2d/%d_%d.png" % (epoch, batch_i))
        plt.close()


    


In [0]:
if __name__ == '__main__':
    gan = Pix2Pix()
    print("training started")
    gan.train(epochs=100, batch_size=3, sample_interval=100)
    print("training ended")

dataloader done
training started


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:75: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:58: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/

training ended


In [0]:
gan.test()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:75: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:58: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


1/1 [==============================] - 1s 1s/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


1/1 [==============================] - 0s 46ms/step


In [0]:
import os
print( os.getcwd() )
print( os.listdir('imagesn2d') )
print( os.listdir('imagesTRecAlln2d') )
print( os.listdir('imagesTestRecn2d') )

/content
['0_73.png', '88_100.png', '52_100.png', '90_100.png', '55_100.png', '0_87.png', '0_164.png', '32_0.png', '32_100.png', '33_100.png', '99_0.png', '0_65.png', '0_34.png', '49_100.png', '0_105.png', '0_84.png', '0_122.png', '0_145.png', '53_0.png', '16_0.png', '95_0.png', '34_100.png', '0_190.png', '0_144.png', '0_89.png', '74_100.png', '0_50.png', '86_100.png', '50_100.png', '0_42.png', '0_133.png', '0_162.png', '22_0.png', '0_167.png', '30_0.png', '0_165.png', '27_100.png', '0_153.png', '0_61.png', '0_92.png', '0_186.png', '67_100.png', '0_45.png', '0_78.png', '57_100.png', '44_0.png', '80_100.png', '5_100.png', '99_100.png', '0_172.png', '79_100.png', '0_101.png', '31_100.png', '0_1.png', '0_136.png', '0_53.png', '18_100.png', '63_0.png', '0_128.png', '66_0.png', '79_0.png', '0_60.png', '0_176.png', '0_0.png', '58_0.png', '0_110.png', '80_0.png', '50_0.png', '85_100.png', '40_0.png', '25_100.png', '11_100.png', '0_191.png', '56_0.png', '19_100.png', '0_88.png', '40_100.png', 

In [0]:
print( os.listdir('gdrive/My Drive') )

['Project proposal.gdoc', '12Certificate.pdf', '10th .pdf', '10th_cert.jpg', '12thmarksheet_Color.pdf', 'mmmmmm.PDF', 'CoverLetter-CBS.gdoc', 'Mrudula_Y_MSCS_Resume1 (1).pdf', 'JavaSetup8u161.exe', 'GroupMessenger1.zip', 'Colab Notebooks', 'MRUDULA_Y_ComputerScienceEng_Resume.docx', 'nips_format.docx', 'project1.gdoc', 'main.ipynb', 'MRUDULA_Y_ComputerScienceEng_Resume (1).gdoc', 'MRUDULA_Y_ComputerScienceEng_Resume.gdoc', 'metric_paper.gdoc', 'Untitled document.gdoc', 'log.csv', 'log_rec.csv', 'logtest.csv', 'log_rec_n2d.csv', 'logn2d.csv', 'logtestn2d.csv']


In [0]:
!zip -r /content/cgan_n2d_train.zip /content/imagesn2d
from google.colab import files
files.download("/content/cgan_n2d_train.zip")    


  adding: content/imagesn2d/ (stored 0%)
  adding: content/imagesn2d/0_73.png (deflated 1%)
  adding: content/imagesn2d/88_100.png (deflated 0%)
  adding: content/imagesn2d/52_100.png (deflated 1%)
  adding: content/imagesn2d/90_100.png (deflated 1%)
  adding: content/imagesn2d/55_100.png (deflated 1%)
  adding: content/imagesn2d/0_87.png (deflated 0%)
  adding: content/imagesn2d/0_164.png (deflated 1%)
  adding: content/imagesn2d/32_0.png (deflated 1%)
  adding: content/imagesn2d/32_100.png (deflated 1%)
  adding: content/imagesn2d/33_100.png (deflated 1%)
  adding: content/imagesn2d/99_0.png (deflated 1%)
  adding: content/imagesn2d/0_65.png (deflated 1%)
  adding: content/imagesn2d/0_34.png (deflated 1%)
  adding: content/imagesn2d/49_100.png (deflated 1%)
  adding: content/imagesn2d/0_105.png (deflated 1%)
  adding: content/imagesn2d/0_84.png (deflated 1%)
  adding: content/imagesn2d/0_122.png (deflated 1%)
  adding: content/imagesn2d/0_145.png (deflated 1%)
  adding: content/image

In [0]:
!zip -r /content/cgan_n2d_testAll.zip /content/imagesTRecAlln2d
from google.colab import files
files.download("/content/cgan_n2d_testAll.zip")   

  adding: content/imagesTRecAlln2d/ (stored 0%)
  adding: content/imagesTRecAlln2d/90_90_rec.png (deflated 1%)
  adding: content/imagesTRecAlln2d/86_86_rec.png (deflated 1%)
  adding: content/imagesTRecAlln2d/6_6_rec.png (deflated 1%)
  adding: content/imagesTRecAlln2d/58_58_rec.png (deflated 1%)
  adding: content/imagesTRecAlln2d/47_47_rec.png (deflated 1%)
  adding: content/imagesTRecAlln2d/95_95_rec.png (deflated 1%)
  adding: content/imagesTRecAlln2d/43_43_rec.png (deflated 1%)
  adding: content/imagesTRecAlln2d/44_44_rec.png (deflated 1%)
  adding: content/imagesTRecAlln2d/52_52_rec.png (deflated 1%)
  adding: content/imagesTRecAlln2d/46_46_rec.png (deflated 1%)
  adding: content/imagesTRecAlln2d/82_82_rec.png (deflated 1%)
  adding: content/imagesTRecAlln2d/84_84_rec.png (deflated 1%)
  adding: content/imagesTRecAlln2d/53_53_rec.png (deflated 1%)
  adding: content/imagesTRecAlln2d/40_40_rec.png (deflated 1%)
  adding: content/imagesTRecAlln2d/41_41_rec.png (deflated 1%)
  adding:

In [0]:
!zip -r /content/cgan_n2d_test.zip /content/imagesTestRecn2d
from google.colab import files
files.download("/content/cgan_n2d_test.zip")    

  adding: content/imagesTestRecn2d/ (stored 0%)
  adding: content/imagesTestRecn2d/90_90_rec.png (deflated 9%)
  adding: content/imagesTestRecn2d/86_86_rec.png (deflated 9%)
  adding: content/imagesTestRecn2d/6_6_rec.png (deflated 9%)
  adding: content/imagesTestRecn2d/58_58_rec.png (deflated 9%)
  adding: content/imagesTestRecn2d/47_47_rec.png (deflated 9%)
  adding: content/imagesTestRecn2d/95_95_rec.png (deflated 8%)
  adding: content/imagesTestRecn2d/43_43_rec.png (deflated 9%)
  adding: content/imagesTestRecn2d/44_44_rec.png (deflated 9%)
  adding: content/imagesTestRecn2d/52_52_rec.png (deflated 9%)
  adding: content/imagesTestRecn2d/46_46_rec.png (deflated 16%)
  adding: content/imagesTestRecn2d/82_82_rec.png (deflated 9%)
  adding: content/imagesTestRecn2d/84_84_rec.png (deflated 15%)
  adding: content/imagesTestRecn2d/53_53_rec.png (deflated 8%)
  adding: content/imagesTestRecn2d/40_40_rec.png (deflated 8%)
  adding: content/imagesTestRecn2d/41_41_rec.png (deflated 9%)
  addin